### Introduction 

### Setup

In [ ]:
import pymongo
import pandas as pd
from pandas.io.json import json_normalize 
from pymongo import MongoClient
import numpy as np 

In [ ]:
 #point the client at mongo URI 
client = MongoClient('mongodb://localhost:27017')
#select database
db = client['tipster']

In [ ]:
# flatten data and put into dataframe 
meetings_collection = db.meetings
data=meetings_collection.find({},{"_id":0})
meeting_runners_df= pd.DataFrame(json_normalize(data,'runners',['meeting_url','meeting_date_time']))
meeting_runners_df['won'] = np.where(meeting_runners_df['place'] == 1.0, 1,0 )

for each horse-date check if winner in any of the previous races  

for each horse get the data of the first win  
for each horse-date horse is a winner if date (strictly) > first win 

In [ ]:

form_collection = db.form
data = form_collection.find({},{"_id":0}) 
form_df = pd.DataFrame(json_normalize(data, 'form',['horse_name']))

import re
strip_country = lambda  x: re.match('(.*)( \([A-Z]*\))', x ).group(1).strip()


first_win_df = pd.DataFrame(form_df[form_df.place == 1.0 ].groupby('horse_name')['meeting_date_time'].min().reset_index())
first_win_df['horse_short_name'] =  first_win_df['horse_name'].apply(strip_country)
 
tmp = pd.merge(first_win_df,meeting_runners_df, left_on='horse_short_name', right_on='horse_name', how='right' )
tmp['winning_form'] = np.where( (tmp['meeting_date_time_x'] >= tmp['meeting_date_time_y']) | 
                               (pd.isnull(tmp['meeting_date_time_x']) ),0,1)   

generate pairs - cartesian product of horses for each meeting
find winner.
if date of win is prior to meeting date update beaten other horse in race in 'x' to true



In [ ]:
#cartesian join get pairs of horses in the same meeting
mdf = meeting_runners_df
mdfl = pd.DataFrame(mdf[['meeting_url','horse_name']])
mdfr = mdfl
x = mdfl.merge(mdfr, on='meeting_url')

In [ ]:
# get rid of single horses
x=x[x.horse_name_x != x.horse_name_y]

# convert pairs into sets ( so (x,y) = (y,x)) 
as_set = lambda row:  frozenset((row['horse_name_x'], row['horse_name_y'])) 
s = x.apply(as_set,axis=1)

X = pd.DataFrame({'pairs':s, 'meetings':x['meeting_url']}) 
X.drop_duplicates(inplace=True)

# sort by meeting-pair in order 
# for each meeting pair
X.groupby('pairs').filter(lambda g: len(g) > 1  ).sort_values(['pairs','meetings'])
 
# find the pair in previous meeting and update has beaten other horse column 

### Data

### Features

which horses in a meeting are winners as of meeting date?  

has a horse in a meeting beaten another horse in the same meeting?

for a given pair of horses which one did better against a third horse?  
    how similar were the conditions? 

time since last race (fitness and value of form)

does the horse need to improve - is it a champion

will form improve?   
margin of victory - ( bigger implies form won't change )  (low fitness + ease => improving form) 

commentary:  
   activity during race  
   finish  
   speed 

is progressively better

above or below average time for course?

raised or lowered in class?

proven at distance 

proven for going

weight

draw - look up advantage of draw in course

type of race - handicap/flat/jumps

### Training

### Predict